# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [4]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [6]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

# Embeddings (N x D)
# normalize_embeddings=True es útil si usarás cosine similarity
embeddings = model.encode(
    passages,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

embeddings.shape, embeddings.dtype

2026-01-05 15:03:16.043407: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767625396.272103      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767625396.338171      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767625396.891377      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767625396.891418      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767625396.891421      55 computation_placer.cc:177] computation placer alr

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Batches:   0%|          | 0/1236 [00:00<?, ?it/s]

((79104, 768), dtype('float32'))

In [7]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [8]:
# código base para FAISS
import faiss
import numpy as np

# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(query_embedding, k=10)

ModuleNotFoundError: No module named 'faiss'

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [9]:
!pip install qdrant-client pymilvus weaviate-client chromadb psycopg2-binary pgvector

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.7/603.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 81.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 98.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 90.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# 1. Inicializar cliente en memoria
client_qdrant = QdrantClient(":memory:")

# 2. Crear colección (Método actualizado para evitar DeprecationWarning)
collection_name = "wikipedia_chunks"
vector_size = embeddings.shape[1]

# Verificamos si existe y la borramos para empezar de cero (equivalente a recreate)
if client_qdrant.collection_exists(collection_name):
    client_qdrant.delete_collection(collection_name)

client_qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

# 3. Preparar los puntos para insertar
points = []
for i, row in chunks_df.iterrows():
    points.append(PointStruct(
        id=i,  # Usamos el índice como ID
        vector=embeddings[i].tolist(),
        payload={"text": row["text"], "doc_id": row["doc_id"], "chunk_id": row["chunk_id"]}
    ))

# Insertar (upsert sigue funcionando igual)
# La advertencia de "Local mode" ignórala, es solo un aviso de rendimiento para datasets grandes
client_qdrant.upsert(
    collection_name=collection_name,
    points=points
)
print("Datos cargados correctamente en Qdrant.")

# 4. Función de búsqueda CORREGIDA
def qdrant_search(query_vec, k=5):
    # En versiones nuevas, a veces .search() da problemas. 
    # Usamos .query_points() que es el método directo a la API de puntos.
    search_result = client_qdrant.query_points(
        collection_name=collection_name,
        query=query_vec.flatten().tolist(), # Nota: el argumento es 'query', no 'query_vector' aquí
        limit=k
    ).points # query_points devuelve un objeto QueryResponse, accedemos a .points
    
    results = []
    for hit in search_result:
        results.append({
            "id": hit.id,
            "score": hit.score,
            "text": hit.payload["text"],
            "metadata": {"doc_id": hit.payload["doc_id"]}
        })
    return results

# Prueba
print("--- Resultados Qdrant (Corregido) ---")
results_qdrant = qdrant_search(query_vec, k=5)
for res in results_qdrant:
    print(f"ID: {res['id']}, Score: {res['score']:.4f}\nTexto: {res['text'][:100]}...\n")

Datos cargados correctamente en Qdrant.
--- Resultados Qdrant (Corregido) ---
ID: 10176, Score: 0.8703
Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...

ID: 1, Score: 0.8618
Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...

ID: 10177, Score: 0.8401
Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...

ID: 37406, Score: 0.8391
Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...

ID: 71872, Score: 0.8386
Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...



## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [13]:
# Instalar la versión ligera de Milvus para uso local
!pip install "pymilvus[milvus-lite]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 36.8 MB/s eta 0:00:00:00:0100:01


In [15]:
# --- PARTE 4: MILVUS (CORREGIDO CON BATCHING) ---
from pymilvus import MilvusClient
from tqdm.auto import tqdm # Barra de progreso opcional pero útil

# 1. Conectar a Milvus Lite
client_milvus = MilvusClient("milvus_demo.db")

# 2. Crear colección
collection_name = "wiki_milvus"

if client_milvus.has_collection(collection_name):
    client_milvus.drop_collection(collection_name)

client_milvus.create_collection(
    collection_name=collection_name,
    dimension=embeddings.shape[1],
    metric_type="COSINE", 
    auto_id=False 
)

# 3. Preparar datos
print("Preparando datos para Milvus...")
data_milvus = []
for i, row in chunks_df.iterrows():
    data_milvus.append({
        "id": i,
        "vector": embeddings[i].tolist(),
        "text": row["text"],
        "doc_id": row["doc_id"]
    })

# 4. Insertar datos POR LOTES (Aquí estaba el error)
batch_size = 5000 # Insertamos de 5000 en 5000 para no saturar la memoria
print(f"Insertando {len(data_milvus)} registros en lotes de {batch_size}...")

for i in tqdm(range(0, len(data_milvus), batch_size)):
    batch = data_milvus[i : i + batch_size]
    client_milvus.insert(collection_name=collection_name, data=batch)

print("Inserción completada exitosamente.")

# 5. Función de búsqueda
def milvus_search(query_vec, k=5):
    search_res = client_milvus.search(
        collection_name=collection_name,
        data=[query_vec.flatten().tolist()],
        limit=k,
        search_params={"metric_type": "COSINE", "params": {}}, 
        output_fields=["text", "doc_id"]
    )
    
    formatted_results = []
    for hit in search_res[0]:
        formatted_results.append({
            "id": hit["id"],
            "score": hit["distance"],
            "text": hit["entity"]["text"],
            "metadata": {"doc_id": hit["entity"]["doc_id"]}
        })
    return formatted_results

# Prueba
print("\n--- Resultados Milvus ---")
results_milvus = milvus_search(query_vec, k=5)
for res in results_milvus:
    print(f"ID: {res['id']} | Score: {res['score']:.4f}\nTexto: {res['text'][:100]}...\n")

Preparando datos para Milvus...
Insertando 79104 registros en lotes de 5000...


  0%|          | 0/16 [00:00<?, ?it/s]

Inserción completada exitosamente.

--- Resultados Milvus ---
ID: 10176 | Score: 0.8703
Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...

ID: 1 | Score: 0.8618
Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...

ID: 10177 | Score: 0.8401
Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...

ID: 37406 | Score: 0.8391
Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...

ID: 71872 | Score: 0.8386
Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...



## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [17]:
import os
import logging
import warnings

# --- CONFIGURACIÓN PARA SILENCIAR LOGS (EJECUTAR PRIMERO) ---
# Silenciar warnings de Python (DeprecationWarning, etc.)
warnings.filterwarnings("ignore")

# Configurar variables de entorno para que el servidor Weaviate sea menos ruidoso
os.environ["LOG_LEVEL"] = "error"
os.environ["WEAVIATE_LOG_LEVEL"] = "error"

# Silenciar los loggers de las librerías de Python
logging.getLogger("weaviate").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)

# --- CÓDIGO PRINCIPAL WEAVIATE ---
import weaviate
from weaviate.classes.config import Configure, Property, DataType

print("Iniciando Weaviate (esto puede tardar unos segundos)...")

# 1. Conectar a instancia embebida
client_wv = weaviate.connect_to_embedded()

# 2. Definir esquema (Collection)
collection_name = "WikiChunk"

# Limpiar si existe (para reiniciar el ejercicio limpio)
if client_wv.collections.exists(collection_name):
    client_wv.collections.delete(collection_name)

# Crear colección con configuración actualizada
chunks_col = client_wv.collections.create(
    name=collection_name,
    vectorizer_config=Configure.Vectorizer.none(), # No usamos vectorizador interno, traemos el nuestro (E5)
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="doc_id", data_type=DataType.INT),
    ]
)

# 3. Insertar datos
print(f"Insertando {len(chunks_df)} documentos...")
# Usamos batch para velocidad
with chunks_col.batch.dynamic() as batch:
    for i, row in chunks_df.iterrows():
        batch.add_object(
            properties={
                "text": row["text"],
                # Aseguramos que sea int nativo de Python para evitar errores de serialización
                "doc_id": int(row["doc_id"]) 
            },
            vector=embeddings[i].tolist()
        )

# 4. Función de búsqueda
def weaviate_search(query_vec, k=5):
    chunks = client_wv.collections.get(collection_name)
    response = chunks.query.near_vector(
        near_vector=query_vec.flatten().tolist(),
        limit=k,
        return_metadata=["distance"]
    )
    
    results = []
    for obj in response.objects:
        results.append({
            "id": obj.uuid,
            # Convertimos distancia a score de similitud (1 - distancia)
            "score": 1 - obj.metadata.distance, 
            "text": obj.properties["text"],
            "metadata": {"doc_id": obj.properties["doc_id"]}
        })
    return results

# Prueba
print("\n--- Resultados Weaviate ---")
results_wv = weaviate_search(query_vec, k=5)
for res in results_wv:
    print(f"Score (Sim): {res['score']:.4f} | Texto: {res['text'][:100]}...")

# Cerrar cliente al terminar (IMPORTANTE)
client_wv.close()

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 243


Iniciando Weaviate (esto puede tardar unos segundos)...


{"action":"load_all_shards","build_git_commit":"","build_go_version":"go1.24.3","build_image_tag":"","build_wv_version":"1.30.5","level":"error","msg":"failed to load all shards: context canceled","time":"2026-01-05T16:38:02Z"}


Insertando 79104 documentos...

--- Resultados Weaviate ---
Score (Sim): 0.8703 | Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...
Score (Sim): 0.8618 | Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...
Score (Sim): 0.8401 | Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...
Score (Sim): 0.8391 | Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...
Score (Sim): 0.8386 | Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...


{"build_git_commit":"","build_go_version":"go1.24.3","build_image_tag":"","build_wv_version":"1.30.5","error":"cannot find peer","level":"error","msg":"transferring leadership","time":"2026-01-05T16:39:21Z"}


## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [18]:
# --- PARTE 6: CHROMA DB ---
import chromadb
from tqdm.auto import tqdm

print("Iniciando ChromaDB...")

# 1. Cliente en memoria (se borra al reiniciar)
client_chroma = chromadb.Client()

# 2. Crear colección
collection_name = "wiki_chroma"
# Borramos si existe para empezar limpio
try:
    client_chroma.delete_collection(collection_name)
except:
    pass 

# "hnsw:space": "cosine" es importante para usar similitud Coseno
collection = client_chroma.create_collection(
    name=collection_name, 
    metadata={"hnsw:space": "cosine"} 
)

# 3. Preparar datos
# Chroma necesita listas separadas
ids = [str(i) for i in chunks_df.index] # Los IDs deben ser strings
embeds_list = embeddings.tolist()
docs_list = chunks_df["text"].tolist()
metas_list = [{"doc_id": int(r["doc_id"]), "chunk_id": int(r["chunk_id"])} for i, r in chunks_df.iterrows()]

# 4. Insertar en lotes (Batching de 5000)
batch_size = 5000
print(f"Insertando {len(ids)} documentos...")

for i in tqdm(range(0, len(ids), batch_size)):
    end = i + batch_size
    collection.add(
        ids=ids[i:end],
        embeddings=embeds_list[i:end],
        documents=docs_list[i:end],
        metadatas=metas_list[i:end]
    )

# 5. Función de búsqueda
def chroma_search(query_vec, k=5):
    results = collection.query(
        query_embeddings=[query_vec.flatten().tolist()],
        n_results=k
    )
    
    formatted = []
    # Chroma devuelve listas de listas
    for i in range(len(results["ids"][0])):
        formatted.append({
            "id": results["ids"][0][i],
            # NOTA: Chroma devuelve DISTANCIA (menor es mejor), no similitud.
            "score": results["distances"][0][i], 
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i]
        })
    return formatted

# Prueba
print("\n--- Resultados Chroma ---")
results_chroma = chroma_search(query_vec, k=5)
for res in results_chroma:
    # Una distancia baja (ej: 0.12) equivale a una similitud alta (0.88)
    print(f"Distancia: {res['score']:.4f} | Texto: {res['text'][:100]}...")

Iniciando ChromaDB...
Insertando 79104 documentos...


  0%|          | 0/16 [00:00<?, ?it/s]


--- Resultados Chroma ---
Distancia: 0.1297 | Texto: Battery tester A battery tester is an electronic device intended for testing the state of an electri...
Distancia: 0.1382 | Texto: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives inform...
Distancia: 0.1599 | Texto: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-ac...
Distancia: 0.1609 | Texto: ils. One was connected via a series resistor to the battery supply. The second was connected to the ...
Distancia: 0.1614 | Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in follo...


## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [20]:
# --- PARTE 7: POSTGRESQL + PGVECTOR ---
import psycopg2
import numpy as np

# Configuración de conexión (Simulada o Real)
# Si estás en Kaggle/Colab, esto probablemente fallará y saltará al 'except'
DB_CONFIG = {
    "host": "localhost",
    "database": "postgres",
    "user": "postgres",
    "password": "mysecretpassword", # Contraseña genérica
    "port": "5432"
}

print("Iniciando Parte 7: PostgreSQL + pgvector...")

try:
    # 1. Intentar conexión
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # 2. Setup (Habilitar extensión y tabla)
    # Esto habilita la matemática vectorial dentro de SQL
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
    cur.execute("DROP TABLE IF EXISTS documents")
    
    dim = embeddings.shape[1] # 768 para E5-base
    cur.execute(f"CREATE TABLE documents (id bigserial PRIMARY KEY, text text, doc_id int, embedding vector({dim}))")
    
    # 3. Insertar datos (Muestra de 100 para probar rápido)
    print("Conexión exitosa. Insertando muestra de datos...")
    data_sql = []
    for i in range(100): 
        row = chunks_df.iloc[i]
        # pgvector recibe listas de python y las convierte automáticamente
        data_sql.append((row["text"], int(row["doc_id"]), embeddings[i].tolist()))
    
    cur.executemany("INSERT INTO documents (text, doc_id, embedding) VALUES (%s, %s, %s)", data_sql)
    conn.commit()

    # 4. Búsqueda SQL
    # El operador <=> calcula la Distancia Coseno
    query_sql = f"""
        SELECT text, doc_id, embedding <=> %s::vector AS distance
        FROM documents
        ORDER BY distance ASC
        LIMIT 5
    """
    cur.execute(query_sql, (query_vec.flatten().tolist(),))
    rows = cur.fetchall()
    
    print("\n--- Resultados PGVector (SQL Real) ---")
    for r in rows:
        print(f"Distancia: {r[2]:.4f} | Texto: {r[0][:100]}...")

    cur.close()
    conn.close()

except Exception as e:
    # ESTO ES LO QUE PROBABLEMENTE VERÁS SI NO TIENES DOCKER/POSTGRES INSTALADO
    print("\n⚠️ AVISO: No se detectó una base de datos PostgreSQL local.")
    print(f"Detalle: {e}")
    print("\n" + "="*50)
    print("RESUMEN TEÓRICO PARA EL INFORME (Parte 7)")
    print("="*50)
    print("Como no hay servidor SQL disponible, aquí tienes la lógica de cómo funciona:")
    print("\n1. ALMACENAMIENTO:")
    print("   Postgres usa un tipo de dato especial 'vector(768)' que guarda el array de floats.")
    print("   Se guarda junto a tus columnas normales (id, texto, fecha) en la misma tabla.")
    print("\n2. BÚSQUEDA (La Query Mágica):")
    print("   SELECT * FROM documents ORDER BY embedding <=> '[0.1, 0.2, ...]' LIMIT 5;")
    print("   - El operador '<=>' calcula la distancia Coseno.")
    print("   - El 'ORDER BY' encuentra los vectores más cercanos.")
    print("\n3. VENTAJAS:")
    print("   - Puedes hacer JOINS: 'Dame documentos similares PERO solo del usuario X'.")
    print("   - No necesitas mover datos de una DB a otra (todo vive en SQL).")

Iniciando Parte 7: PostgreSQL + pgvector...

⚠️ AVISO: No se detectó una base de datos PostgreSQL local.
Detalle: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


RESUMEN TEÓRICO PARA EL INFORME (Parte 7)
Como no hay servidor SQL disponible, aquí tienes la lógica de cómo funciona:

1. ALMACENAMIENTO:
   Postgres usa un tipo de dato especial 'vector(768)' que guarda el array de floats.
   Se guarda junto a tus columnas normales (id, texto, fecha) en la misma tabla.

2. BÚSQUEDA (La Query Mágica):
   SELECT * FROM documents ORDER BY embedding <=> '[0.1, 0.2, ...]' LIMIT 5;
   - El operador '<=>' calcula la distancia Coseno.
   - El 'ORDER BY' encuentra los vectores más cercanos.

3. VENTAJAS:
   - Puedes hacer JOINS: 'Dame documentos